## Projet 10 : Application de recommandation de contenu

Dans ce notebook, nous allons effectuer l'analyse exploratoire des données (EDA) pour déterminer l'organisation des données brutes et leur composition. Cette analyse nous permettra de mieux comprendre les structures de données, d'identifier les tendances et les patterns, et de préparer les données pour la phase de développement de notre MVP.

### 1. Import

#### 1.1 Import des packages

Voici les libraries que nous allons utiliser
- ``os`` : Module Python pour interagir avec le système d'exploitation.
- ``pandas`` : Bibliothèque pour la manipulation de données.
- ``numpy`` : Supporte les tableaux et matrices multidimensionnels.
- ``matplotlib.pyplot`` : Bibliothèque de graphiques 2D.
- ``seaborn`` : Visualisation de données basée sur matplotlib.
- ``pickle`` : Module pour sérialiser et désérialiser les structures d'objets Python.

In [1]:
import os 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle 


#### 1.2 Import des données

<p align="center">
  <img src="../resources/img/globo_logo.png" />
</p>

Les données que nous allons utiliser proviennent d'un vaste ensemble de données de recommandations de news, comprenant les logs d'interactions des utilisateurs du site globo.com, l'un des sites de nouvelles les plus populaires au Brésil. Ces données offrent un aperçu précieux des comportements des utilisateurs et des tendances de consommation de nouvelles.

Lien source des données : https://www.kaggle.com/datasets/gspmoreira/news-portal-user-interactions-by-globocom/data"

In [2]:
data_path = Path("../data/raw/globocom/")
all_clicks = Path("../data/raw/globocom/clicks/")

### 2. Première Analyse 

Maintenant, nous allons explorer les données pour avoir une première visualisation de leur contenu. Cela nous permettra de comprendre mieux la structure des données, les types de valeurs qu'elles contiennent et les relations potentielles entre différentes variables. Cette étape est essentielle pour déterminer comment nous pouvons utiliser au mieux ces données pour répondre à nos questions de recherche.

#### 2.1 Structure des données

In [3]:
# liste des tous les fichiers dans le dossier en excluant le fichier .DS_Store
files = [f.name for f in data_path.iterdir() if f.name != '.DS_Store']

# affichage des fichiers dans le dossier
for file in files:
    print(f"📂 {file}")

📂 clicks_sample.csv
📂 articles_embeddings.pickle
📂 articles_metadata.csv
📂 clicks


### 2.2 Affichage des données et analyse du contenu

``click_samples.csv``

In [4]:
df_sample=pd.read_csv(str(data_path) + "/clicks_sample.csv")
print(f"Nombre de lignes: {df_sample.shape[0]}, Nombre de colonnes: {df_sample.shape[1]}")
df_sample.head()

Nombre de lignes: 1883, Nombre de colonnes: 12


,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2


Cette DataFrame répresente une sample des données qui sont contenues dans le dossier ``clicks``.
Chaque ligne répresente les interactions des utilisateurs avec le site.

- `user_id` : L'ID de l'utilisateur qui a effectué le clic.

- `session_id` : L'ID de la session pendant laquelle le clic a été effectué.

- `session_start` : Le moment du début de la session.

- `session_size` : La taille de la session.

- `click_article_id` : L'ID de l'article qui a été cliqué.

- `click_timestamp` : Le timestamp du clic.

- `click_environment` : L'environnement dans lequel le clic a été effectué.

- `click_deviceGroup` : Le groupe de dispositifs à partir duquel le clic a été effectué.

- `click_os` : Le système d'exploitation à partir duquel le clic a été effectué.

- `click_country` : Le pays à partir duquel le clic a été effectué.

- `click_region` : La région à partir de laquelle le clic a été effectué.

- `click_referrer_type` : Le type de référent pour le clic.


In [11]:
clicks_path = data_path / 'clicks'
file_count = len(list(Path(clicks_path).glob('*')))
print("Numero des fichiers dans le dossier clicks:", file_count)

Numero des fichiers dans le dossier clicks: 385


In [15]:
# liste des fichiers csv dans le dossier clicks
csv_files = list(clicks_path.glob('*.csv'))

# affichage de l'element 51 
print(csv_files[50].name)

clicks_hour_112.csv


Dans le dossier nommé "clicks", nous disposons de 385 fichiers. Chaque fichier représente une heure de données relatives aux clics effectués par les utilisateurs. 

``articles_metadata.csv``

In [5]:
df_sample=pd.read_csv(str(data_path) + "/articles_metadata.csv")
print(f"Nombre de lignes: {df_sample.shape[0]}, Nombre de colonnes: {df_sample.shape[1]}")
df_sample.head()

Nombre de lignes: 364047, Nombre de colonnes: 5


,article_id,category_id,created_at_ts,publisher_id,words_count
0,0,0,1513144419000,0,168
1,1,1,1405341936000,0,189
2,2,1,1408667706000,0,250
3,3,1,1408468313000,0,230
4,4,1,1407071171000,0,162


Cette Dataframe contient les metadata des $364047$ articles publiées dans les site.
- ``article_id`` : L'ID de l'article.

- ``category_id`` : L'ID de la catégorie à laquelle appartient l'article.

- ``created_at_ts`` : Le timestamp de la création de l'article.

- ``publisher_id`` : L'ID de l'éditeur de l'article.

- ``words_count`` : Le nombre de mots dans l'article.

``Articles_embeddings.pickle``

In [7]:
# Ouvrir le fichiers pickle et afficher les 5 premiers lignes
with open(data_path / 'articles_embeddings.pickle', 'rb') as f:
    data = pickle.load(f)

df = pd.DataFrame(data)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,-0.161183,-0.957233,-0.137944,0.050855,0.830055,0.901365,-0.335148,-0.559561,-0.500603,0.165183,...,0.321248,0.313999,0.636412,0.169179,0.540524,-0.813182,0.286870,-0.231686,0.597416,0.409623
1,-0.523216,-0.974058,0.738608,0.155234,0.626294,0.485297,-0.715657,-0.897996,-0.359747,0.398246,...,-0.487843,0.823124,0.412688,-0.338654,0.320787,0.588643,-0.594137,0.182828,0.397090,-0.834364
2,-0.619619,-0.972960,-0.207360,-0.128861,0.044748,-0.387535,-0.730477,-0.066126,-0.754899,-0.242004,...,0.454756,0.473184,0.377866,-0.863887,-0.383365,0.137721,-0.810877,-0.447580,0.805932,-0.285284
3,-0.740843,-0.975749,0.391698,0.641738,-0.268645,0.191745,-0.825593,-0.710591,-0.040099,-0.110514,...,0.271535,0.036040,0.480029,-0.763173,0.022627,0.565165,-0.910286,-0.537838,0.243541,-0.885329
4,-0.279052,-0.972315,0.685374,0.113056,0.238315,0.271913,-0.568816,0.341194,-0.600554,-0.125644,...,0.238286,0.809268,0.427521,-0.615932,-0.503697,0.614450,-0.917760,-0.424061,0.185484,-0.580292


Cette dataframe contient une matrice NumPy qui renferme les embeddings des articles. Ces embeddings sont des vecteurs de 250 dimensions qui ont été entraînés sur le texte et les métadonnées en utilisant le module CHAMELEON ACR. Pour plus de détails sur ce module, vous pouvez consulter le lien suivant : https://arxiv.org/abs/1808.00076. Les embeddings sont des représentations vectorielles qui capturent les caractéristiques sémantiques des articles. 

Nous pouvons utiliser une technique de réduction de dimensionnalité, notamment le t-SNE comme indiqué sur le site source des données : https://www.kaggle.com/datasets/gspmoreira/news-portal-user-interactions-by-globocom#clicks_sample.csv, pour visualiser les embeddings colorée par catégories.

In [8]:
#from sklearn.decomposition import PCA
#from sklearn.manifold import TSNE
#import matplotlib.pyplot as plt

# Use PCA to reduce the dimensionality of the data to 50
#pca = PCA(n_components=50)
#data_pca = pca.fit_transform(df)

# Use t-SNE to reduce the dimensionality of the PCA data to 2D
#tsne = TSNE(n_components=2, random_state=0)
#data_2d = tsne.fit_transform(data_pca)

# Create a scatter plot of the 2D data
#plt.figure(figsize=(10, 10))
#plt.scatter(data_2d[:, 0], data_2d[:, 1])
#plt.show()

KeyboardInterrupt: 